In [6]:
#train with seperate encoder and predictor models bc I can't get transfer learning to work
import math, random, sys
sys.path.insert(0, '/home/marcase/hgraph2graph/')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
import networkx as nx
import rdkit
import numpy as np
import argparse
import os
from tqdm.auto import tqdm
import pickle
from hgraph import *
from hgraph.inc_graph import *
from hgraph.encoder import *
import matplotlib.pyplot as plt
from hgraph.predict import HierPredict

In [10]:
vocab = '/scratch/gthurber_root/gthurber0/marcase/preprocess_mono/preprocess_generated_real/vocab26oct22.txt'
vocab = [x.strip("\r\n ").split() for x in open(vocab)]
vocab = PairVocab(vocab)
# vocab.vocab[21][0]
# vocab.hvocab

In [16]:
class Args:
    train = '/scratch/gthurber_root/gthurber0/marcase/preprocess_mono/preprocess_generated_real/train/'
    train_labels = '/scratch/gthurber_root/gthurber0/marcase/preprocess_mono/preprocess_generated_real/train_labels/'
    test = '/scratch/gthurber_root/gthurber0/marcase/preprocess_mono/preprocess_generated_real/test_data/'
    test_labels = '/scratch/gthurber_root/gthurber0/marcase/preprocess_mono/preprocess_generated_real/test_labels/'
    vocab = vocab
    save_dir = 'test/'
    atom_vocab = common_atom_vocab
    load_model = None
    seed = 7
    rnn_type = 'LSTM'
    hidden_size=250
    embed_size=250
    batch_size=32
    latent_size=32
    depthT=15
    depthG=15
    diterT=1
    diterG=3
    dropout=0.5
    lr = 1e-3
    clip_norm=5.0
    step_beta=0.001
    max_beta=1.0
    warmup=10000
    kl_anneal_iter=2000
    epoch=2000
    anneal_rate=0.9
    anneal_iter=25000
    print_iter=50
    save_iter=1000000
    model = '/scratch/gthurber_root/gthurber0/marcase/preprocess_mono/preprocess_generated_real/ckpt/pretrained/model.ckpt.340000'
    load_model = True
    nsample = 1
    label_size = 2
    max_AA = 15
    max_nodes = 100
    max_edges = 200
    max_sub_nodes = 200
    
args=Args()

encoder = HierVAE(args).cuda()

In [17]:
model = HierPredict(args).cuda()
model

HierPredict(
  (ff1): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (ff2): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (ff3): Sequential(
    (0): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [18]:
torch.manual_seed(args.seed)
random.seed(args.seed)

for param in encoder.parameters():
    if param.dim() == 1:
        nn.init.constant_(param, 0)
    else:
        nn.init.xavier_normal_(param)
        
for param in model.parameters():
    if param.dim() == 1:
        nn.init.constant_(param, 0)
    else:
        nn.init.xavier_normal_(param)        

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)
scheduler = lr_scheduler.ExponentialLR(optimizer, args.anneal_rate)

if args.load_model:
    print('continuing from checkpoint ' + args.model)
    model_state, optimizer_state, total_step, beta = torch.load(args.model)
    
    encoder.load_state_dict(model_state)

else:
    total_step = beta = 0

param_norm = lambda m: math.sqrt(sum([p.norm().item() ** 2 for p in m.parameters()]))
grad_norm = lambda m: math.sqrt(sum([p.grad.norm().item() ** 2 for p in m.parameters() if p.grad is not None]))



continuing from checkpoint /scratch/gthurber_root/gthurber0/marcase/preprocess_mono/preprocess_generated_real/ckpt/pretrained/model.ckpt.340000


In [19]:
meters = np.array([])
meters_list = list(meters)
validation_list = list()
total_step = 0

In [20]:
for epoch in range(args.epoch):
    random.seed(args.seed)
    dataset_x = DataFolder(args.train, args.batch_size,shuffle = False)
    dataset_y = DataFolder(args.train_labels, args.batch_size,shuffle = False)
    dataset_x.data_files = ['tensors-'+str(i)+'.pkl' for i in range(len(dataset_x.data_files))]
    dataset_y.data_files = ['tensors_labels-'+str(i)+'.pkl' for i in range(len(dataset_y.data_files))]
    model.train()
    for batch_x,batch_y in zip(dataset_x,dataset_y):
        total_step += 1
        model.zero_grad()
        latent = encoder(*batch_x, beta=beta,decode=False) 
        y_pred = model(latent)
        y_true = torch.Tensor([int(y) for y in batch_y]).cuda()
        y_true = y_true.type(torch.LongTensor).cuda()
        loss = criterion(y_pred,y_true)
        accuracy = torch.sum(torch.argmax(y_pred, dim=1).cuda() == y_true)/len(y_true)
        # loss = Variable(loss, requires_grad = True)
        loss.backward()
        optimizer.step()

        meters = np.array([loss.item(),accuracy.cpu()])
        meters_list.append(meters)

        if total_step % args.print_iter == 0:
            print("[%d] Beta: %.3f, loss: %.3f, accuracy: %.3f, PNorm: %.2f, GNorm: %.2f" % (total_step, beta, meters[0], meters[1], param_norm(model), grad_norm(model)))
            sys.stdout.flush()
            meters *= 0
        
        if total_step % args.save_iter == 0:
            ckpt = (model.state_dict(), optimizer.state_dict(), total_step, beta)
            torch.save(ckpt, os.path.join(args.save_dir, f"model.ckpt.{total_step}"))

        if total_step % args.anneal_iter == 0:
            scheduler.step()
            print("learning rate: %.6f" % scheduler.get_lr()[0])

        if total_step >= args.warmup and total_step % args.kl_anneal_iter == 0:
            beta = min(args.max_beta, beta + args.step_beta)
    
    #"validation" set
    model.eval()
    dataset_x = DataFolder(args.test, args.batch_size,shuffle = False)
    dataset_y = DataFolder(args.test_labels, args.batch_size,shuffle = False)
    dataset_x.data_files = ['tensors-'+str(i)+'.pkl' for i in range(len(dataset_x.data_files))]
    dataset_y.data_files = ['tensors_labels-'+str(i)+'.pkl' for i in range(len(dataset_y.data_files))]
    random.seed()
    i=0
    accuracy_list = list()
    for batch_x,batch_y in zip(dataset_x,dataset_y):
        batch_x0 = batch_x
        batch_y0 = batch_y
        latent = encoder(*batch_x, beta=beta,decode=False) 
        y_pred = model(latent)
        y_pred = torch.argmax(y_pred, dim=1)
        y_true = torch.Tensor([int(y) for y in batch_y0]).cuda()
        y_true = y_true.type(torch.LongTensor).cuda()
        accuracy_list.append((torch.sum(y_pred == y_true)/len(y_pred)).item())
        i += 1


    print('Accuracy on validation set: %.3f' % np.average(accuracy_list))
    validation_list.append((torch.sum(y_pred == y_true)/len(y_pred)).item())

[50] Beta: 0.165, loss: 0.970, accuracy: 0.312, PNorm: 8.13, GNorm: 1.74
[100] Beta: 0.165, loss: 1.105, accuracy: 0.312, PNorm: 8.05, GNorm: 2.78
[150] Beta: 0.165, loss: 0.791, accuracy: 0.500, PNorm: 7.98, GNorm: 1.46
[200] Beta: 0.165, loss: 0.759, accuracy: 0.531, PNorm: 7.93, GNorm: 1.05
[250] Beta: 0.165, loss: 0.717, accuracy: 0.531, PNorm: 7.89, GNorm: 0.83
[300] Beta: 0.165, loss: 0.648, accuracy: 0.594, PNorm: 7.85, GNorm: 1.15
[350] Beta: 0.165, loss: 0.695, accuracy: 0.562, PNorm: 7.83, GNorm: 0.70
[400] Beta: 0.165, loss: 0.850, accuracy: 0.406, PNorm: 7.80, GNorm: 1.46
[450] Beta: 0.165, loss: 0.717, accuracy: 0.531, PNorm: 7.78, GNorm: 0.64
[500] Beta: 0.165, loss: 0.709, accuracy: 0.500, PNorm: 7.77, GNorm: 0.60
[550] Beta: 0.165, loss: 0.667, accuracy: 0.531, PNorm: 7.75, GNorm: 0.61
[600] Beta: 0.165, loss: 0.685, accuracy: 0.594, PNorm: 7.74, GNorm: 0.69
[650] Beta: 0.165, loss: 0.742, accuracy: 0.500, PNorm: 7.73, GNorm: 0.74
[700] Beta: 0.165, loss: 0.640, accurac

KeyboardInterrupt: 

In [15]:
dataset_x = DataFolder(args.train, args.batch_size,shuffle = False)
dataset_x.data_files = ['tensors-'+str(i)+'.pkl' for i in range(len(dataset_x.data_files))]
dataset_x.data_files

['tensors-0.pkl',
 'tensors-1.pkl',
 'tensors-2.pkl',
 'tensors-3.pkl',
 'tensors-4.pkl',
 'tensors-5.pkl',
 'tensors-6.pkl',
 'tensors-7.pkl',
 'tensors-8.pkl',
 'tensors-9.pkl',
 'tensors-10.pkl',
 'tensors-11.pkl',
 'tensors-12.pkl',
 'tensors-13.pkl',
 'tensors-14.pkl',
 'tensors-15.pkl',
 'tensors-16.pkl',
 'tensors-17.pkl',
 'tensors-18.pkl',
 'tensors-19.pkl',
 'tensors-20.pkl',
 'tensors-21.pkl',
 'tensors-22.pkl',
 'tensors-23.pkl',
 'tensors-24.pkl',
 'tensors-25.pkl',
 'tensors-26.pkl',
 'tensors-27.pkl',
 'tensors-28.pkl',
 'tensors-29.pkl',
 'tensors-30.pkl',
 'tensors-31.pkl',
 'tensors-32.pkl',
 'tensors-33.pkl']

In [14]:
y_pred = model(latent)
y_pred

tensor([[0.1701, 0.8299],
        [0.1210, 0.8790],
        [0.4298, 0.5702],
        [0.4205, 0.5795],
        [0.3272, 0.6728],
        [0.1508, 0.8492],
        [0.2056, 0.7944],
        [0.1318, 0.8682],
        [0.2318, 0.7682],
        [0.2574, 0.7426],
        [0.3937, 0.6063],
        [0.3240, 0.6760],
        [0.2951, 0.7049],
        [0.2564, 0.7436],
        [0.3607, 0.6393],
        [0.2876, 0.7124],
        [0.4141, 0.5859],
        [0.2326, 0.7674],
        [0.1597, 0.8403],
        [0.3544, 0.6456],
        [0.2972, 0.7028],
        [0.3681, 0.6319],
        [0.3508, 0.6492],
        [0.3615, 0.6385],
        [0.3248, 0.6752],
        [0.1423, 0.8577],
        [0.1934, 0.8066],
        [0.1963, 0.8037],
        [0.3888, 0.6112],
        [0.2786, 0.7214],
        [0.2361, 0.7639],
        [0.3717, 0.6283]], device='cuda:0', grad_fn=<SoftmaxBackward0>)